In [12]:
pip install openfermionpyscf

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [26]:
%env CUDAQ_LOG_LEVEL=info

env: CUDAQ_LOG_LEVEL=info


In [27]:
%env

{'SHELL': '/bin/bash',
 'NV_LIBCUBLAS_VERSION': '11.11.3.6-1',
 'NVIDIA_VISIBLE_DEVICES': 'all',
 'UCX_INSTALL_PREFIX': '/usr/local/ucx',
 'SGE_RERUN_JOB': '0',
 'SGE_O_SHELL': '/bin/bash',
 'HISTCONTROL': 'ignoredups',
 'CUQUANTUM_INSTALL_PREFIX': '/opt/nvidia/cuquantum',
 'HOSTNAME': 'g0225.abci.local',
 'HISTSIZE': '1000',
 'SINGULARITY_NAME': 'cuda-quantum_0.5.0.sif',
 'SGE_O_HOME': '/home/acf16133sf',
 'SGE_ROOT': '/home/system/uge/latest',
 'JOB_SCRIPT': 'QRLOGIN',
 'SGE_ACCOUNT': 'gae50954',
 'SGE_JOB_SERVICE_TYPE': 'On-demand',
 'NVIDIA_REQUIRE_CUDA': 'cuda>=11.8 brand=tesla,driver>=470,driver<471 brand=unknown,driver>=470,driver<471 brand=nvidia,driver>=470,driver<471 brand=nvidiartx,driver>=470,driver<471 brand=geforce,driver>=470,driver<471 brand=geforcertx,driver>=470,driver<471 brand=quadro,driver>=470,driver<471 brand=quadrortx,driver>=470,driver<471 brand=titan,driver>=470,driver<471 brand=titanrtx,driver>=470,driver<471',
 'NV_NVTX_VERSION': '11.8.86-1',
 'MODULES_LMCON

In [28]:
import cudaq

In [29]:
geometry = [('H', (0., 0., 0.)), ('H', (0., 0., .7474))]

In [30]:
molecule, data = cudaq.chemistry.create_molecular_hamiltonian(
    geometry, 'sto-3g', 1, 0)

In [31]:
electron_count = data.n_electrons
qubit_count = 2 * data.n_orbitals

In [32]:
qubit_count

4

In [33]:
kernel, angles = cudaq.make_kernel(list)
qubits = kernel.qalloc(qubit_count)

In [34]:
# Prepare the Hartree Fock State.
kernel.x(qubits[0])
kernel.x(qubits[1])

In [35]:
# Adds parameterized gates based on the UCCSD ansatz.
cudaq.kernels.uccsd(kernel, qubits, angles, electron_count, qubit_count)

In [36]:
parameter_count = cudaq.kernels.uccsd_num_parameters(electron_count,
                                                     qubit_count)

optimizer = cudaq.optimizers.COBYLA()

energy, parameters = cudaq.vqe(kernel,
                               molecule,
                               optimizer,
                               parameter_count=parameter_count)


In [37]:
print(energy)

-1.1371756094989427
